# 章節 15：維度處理

## 15.10 預測分佈

這邊用四個相關的問題介紹隱藏物種問題。我們已經解答前兩個問題，分別是計算物種數量的後驗分佈以及每個物種的流行程度。另外兩個問題是：

- 如果我們計畫收集更多的測序片段（read），我們可以預測有多少新的物種被探索到呢？
- 需要多少額外的測序片段才能將觀察到物種的比例增加到給定的門檻值？

為了回答上述的預測問題，我們可以用後驗分佈去模擬未來可能發生的事件，並且計算不同物種數量的預測分佈，我們可能看到總和一部分。（TODO）

這些模擬方法的核心如下：
1. 從後驗分佈選折一個物種數量 n
1. 用 Dirichlet 分佈選擇每種物種的流行程度，包含可能看不到的物種
1. 亂數取樣未來的觀察序列
1. 計算新物種的數量, num_new, 作為額外測序片段數量的函數, k
1. 重複前面步驟累積產生 num_new 和 k 的聯合分佈

以下是 num_new 和 k 的聯合分佈的程式碼。RunSimulation 代表執行一次模擬：

<pre>
# class Subject
    def RunSimulation(self, num_reads):
        m, seen = self.GetSeenSpecies()
        n, observations = self.GenerateObservations(num_reads)
        curve = []
        for k, obs in enumerate(observations):
            seen.add(obs)
            num_new = len(seen) - m
            curve.append((k+1, num_new))
            
        return curve
</pre>

- num_reads 是有多少額外的測序片段來做模擬。
- m 是有多少已經觀察到的數量。是對每個物種帶有唯一名稱的字串的集合。
- n 是後驗分佈的隨機變數
- observations 是每個物種名稱的隨機取樣序列

每一次迭代，我們加上新被觀察的物種到 seen 集合，並且記錄目前的測序片段數量以及新觀察到的物種數量。RunSimulation 的結果是一個稀釋性曲線（rarefaction curve），用一序列測序片段數量與新物種的配對構成。

在我們看結果之前，先介紹 GetSeenSpecies 方法和 GenerateObservations 方法。
<pre>
#class Subject
    def GetSeenSpecies(self):
        names = self.GetNames()
        m = len(names)
        seen = set(SpeciesGenerator(names, m))
        return m, seen
</pre>

GetNames 方法回傳一序列出現在資料集的物種名稱，但對很多主體來說，這些名字不是唯一的。所以這邊用 SpeciesGenerator 方法來對每個名稱給定一個序號：

<pre>
def SpeciesGenerator(names, num): 
    i=0
    for name in names:
        yield '%s-%d' % (name, i)
        i += 1
        
    while i < num:
        yield 'unseen-%d' % i
        i += 1
</pre>

輸入一個物種名稱像是 Corynebacterium，SpeciesGenerator 會回傳 Corynebacterium-1。當一序列的名稱已經用完了，便會回傳類似 unseen-62 的字串。

以下是 GenerateObservations 方法：
<pre>
# class Subject
    def GenerateObservations(self, num_reads):
        n, prevalences = self.suite.SamplePosterior()
        names = self.GetNames()
        name_iter = SpeciesGenerator(names, n)
        d = dict(zip(name_iter, prevalences))
        cdf = thinkbayes.MakeCdfFromDict(d)
        observations = cdf.Sample(num_reads)
        return n, observations
</pre>

- num_reads 是有多少額外的測序片段來做產生
- n 和 prevalences 為後驗分佈的隨機取樣
- cdf 是一個 Cdf 物件，為物種名稱包含未觀察的物種的累積機率分佈。用 Cdf 易於抽樣一序列的物種名稱。

最後是 SamplePosterior 方法與 SamplePrevalences 方法：

<pre>
# class Species2

    def SamplePosterior(self):
        pmf = self.DistOfN()
        n = pmf.Random()
        prevalences = self.SamplePrevalences(n)
        return n, prevalences

    And , which generates a sample of prevalences condi- tioned on n:

    def SamplePrevalences(self, n):
        params = self.params[:n]
        gammas = numpy.random.gamma(params)
        gammas /= gammas.sum()
        return gammas

</pre>
        
SamplePrevalences 方法取樣一個基於 n 個物種的流行程度分佈。我們在 15.4 小節看到用此演算法在 Dirichlet 分佈做亂數取樣。

In [1]:
#TODO

上圖為對主體 B1242 做 100 次模擬的稀釋性曲線（rarefaction curve）結果。曲線經過「抖動」（jittered）;也就是有將曲線做亂數平移，這樣才不會重疊到。透過檢查模擬結果，我們可以預測額外增加 400 個以上的測序片段，我們可能觀察到 2-6 種新的物種。